In [3]:
from fileToolbox import *
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV

In [4]:
def warn(*args, **kwargs):
    pass


import warnings

warnings.warn = warn
warnings.filterwarnings("ignore", category=UserWarning)

In [5]:
set_of_person_ids = set(train_df["person_id"])
list_of_unique_person_ids = list(set_of_person_ids) # osoby to będą moje x

In [8]:
set_of_movie_ids = set(train_df["movie_id"])
list_of_unique_movie_ids = list(set_of_person_ids) # osoby to będą moje x

In [7]:
tmp_p = set(task_df["person_id"])
tmp_m = set(task_df["movie_id"])

Sprawdzenie czy napewno wszystkie person_id i movie_id z taska występują w trainie

In [9]:
for person in set_of_person_ids:
    if person not in tmp_p:
        print(person)

In [10]:
for movie in set_of_movie_ids:
    if movie not in tmp_m:
        print(movie)

Dla każdego filmu generuje n elementowy wektor cech

In [12]:
N = 10 # to będzie hiperparametr

In [13]:
M = len(set_of_movie_ids)
R = len(set_of_person_ids)

In [16]:
print(M)
print(R)

200
358


In [14]:
import random
# losowa inicjalizacja list p - parametrów i x - wejść z przedziału <-1, 1>
X_INPUTS = [[random.uniform(-1, 1) for j in range(N+1)] for i in range(M)] # parametrów jest n+1 bo stała
PARAMS = [[random.uniform(-1, 1) for j in range(R)] for i in range(N)]

In [29]:
def isMovieRatedByPerson(movie, person):
    df_person = filterDataFrame("person_id", person, train_df)
    list_of_movies = list(df_person["movie_id"]))
    if movie in list_of_movies:
        return 1
    else:
        return 0

In [2]:
# def main_function(p, x, y, alfa, beta, num_of_epochs): # alfa - współczynnik uczenia do P, beta - współczynnik uczenia do X
#     for epoch in range(num_of_epochs):
#         for r, p in enumerate(PARAMS):
#             for m in range(len(movies)): # m - liczba filmów, p - parametry da r-tej ososby
#                 for n in range(len(p)):
#                     p[n] = p[n] - alfa * derivative_P(x[m], y, p, n) # poprawiam naraz w jednej iteracji x i p
#                     x[m][n] = x[n] - beta * derivative_X(x[m], y, p, n)

In [ ]:
def main_function(p, x, y, alfa, beta, num_of_epochs): # alfa - współczynnik uczenia do P, beta - współczynnik uczenia do X
    for epoch in range(num_of_epochs):
        for r, p in enumerate(PARAMS): # r - indeks osób
            for m, x in enumerate(X_INPUTS): # m - indeks filmów
                for n in range(len(p)):
                    # poprawiam naraz w jednej iteracji x i p
                    p[n] = p[n] - alfa * derivative_P(X_INPUTS, Y_RESULTS, p, n, r) # UWAGA parametrów jest n+1
                    x[n] = x[n] - beta * derivative_X(x, Y_RESULTS, PARAMS, n, m)
    print(Q(X_INPUTS, PARAMS, Y_RESULTS)) # błąd liczymy po całej przestrzeni

In [1]:
def Q(X, P, Y):
    res = 0.0
    for r, p in enumerate(P): # r - indeks osób
            for m, x in enumerate(X):
                res += (calc_output(x, p) - Y[r][m]) ** 2
    return res / 2 

In [3]:
def derivative_P(X, Y, P, i, r):
    res = 0.0
    for m in range(len(X)): # przechodzimy po filmach m
        if i ==0: # dla p0 bez mnożenia z X[m][i]
            res += (calc_output(X[m], p) - Y[m][r])
        else:
            res += (calc_output(X[m], p) - Y[m][r]) * X[m][i]
    return res  # prawdopodobnie tu powinno być (1 / (len(X))) * res

In [ ]:
def derivative_X(X, Y, p, i, m):
    res = 0.0
    for r in range(len(P)): # przechodzimy po osobach r
        res += (calc_output(X, p[r]) - Y[m][r]) * p[r][i]
    return res  # prawdopodobnie tu powinno być (1 / (len(X))) * res

In [4]:
def calc_output(x, p):
    summ = p[0]
    for n in range(1, len(p)):
        summ = summ + x[n] * p[n]
    return summ